## CNN_default

In [ ]:
### CNN 기본

import numpy as np
import seaborn as sns
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import tensorflow as tf

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#코랩 마운트
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd "/content/drive/My Drive/face"

In [ ]:
#압축해제
!unzip -qq "/content/drive/MyDrive/real-vs-fake.zip"

In [ ]:
# eda를 위해 데이터프레임 저장
path = '/content/drive/My Drive/face/'

dataset = {"image_path":[],"img_status":[],"where":[]}
for where in os.listdir(path):
    for status in os.listdir(path+"/"+where):
        for image in glob.glob(path+where+"/"+status+"/"+"*.jpg"):
            dataset["image_path"].append(image)
            dataset["img_status"].append(status)
            dataset["where"].append(where)
dataset = pd.DataFrame(dataset)
dataset

In [ ]:
#막대 그래프 plot
import seaborn as sns
print(f"Real: {real},\nFake: {fake}\n")
sns.countplot(dataset["img_status"])
plt.show()

In [ ]:
#데이터 경로 지정 등 설정
from tensorflow.keras import layers, models, backend, utils, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator()

train_generator = image_gen.flow_from_directory(
    path + 'train/',
    target_size=(32, 32),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)

valid_generator = image_gen.flow_from_directory(
    path + 'valid/',
    target_size=(32, 32),
    batch_size=64,
    color_mode='grayscale',
    class_mode='binary'
)

test_generator = image_gen.flow_from_directory(
    path + 'test/',
    target_size=(32, 32),
    batch_size=64,
    color_mode='grayscale',
    shuffle = False,
    class_mode='binary'
)

In [ ]:
#구조 생성
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3),activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

In [ ]:
#모델 학습
train_steps = 100000//64
valid_steps = 20000//64

hist = model.fit_generator(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_steps
)

In [ ]:
#acc 및 loss plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate([ "accuracy", "loss"]):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
#test 평가
model.evaluate(test_generator)

## CNN_6Blocks

In [ ]:
### 6개 블록 CNN 


import os
import tensorflow as tf
import keras

from tensorflow.keras import layers, models, backend, utils, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#코랩 마운트
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
#cd /content/drive/MyDrive/face

In [ ]:
#압축해제
#!unzip -qq "/content/drive/MyDrive/real-vs-fake.zip"

In [ ]:
#개수 확인(14만개 사용)
from glob import glob
filepaths1 = list(glob('/content/drive/MyDrive/face/train/real/*.jpg'))
filepaths2 = list(glob('/content/drive/MyDrive/face/test/real/*.jpg'))
filepaths3 = list(glob('/content/drive/MyDrive/face/valid/real/*.jpg'))
print(len(filepaths1), len(filepaths2), len(filepaths3))

In [ ]:
#데이터 경로 지정 등 설정
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator()

path = '/content/drive/My Drive/face/'

train_generator = image_gen.flow_from_directory(
    path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    class_mode='binary'
)

valid_generator = image_gen.flow_from_directory(
    path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    class_mode='binary'
)

test_generator = image_gen.flow_from_directory(
    path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    shuffle = False,
    class_mode='binary'
)

In [ ]:
#구조 생성
model = models.Sequential()

model.add(
    layers.Conv2D(32, (3, 3),activation='relu', input_shape=(224, 224, 3))
)

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(
    optimizer = 'adam', 
    loss = 'binary_crossentropy', 
    metrics = [
        'accuracy'
    ]
)

model.summary()

In [ ]:
#모델 학습
train_steps = 100000//64
valid_steps = 20000//64

hist = model.fit(
    train_generator,
    epochs=20,
    validation_data=valid_generator,
    validation_steps=valid_steps
)

In [ ]:
#acc 및 loss plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate([ "accuracy", "loss"]):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
#test 평가
model.evaluate(test_generator)

## VGG

In [ ]:
## VGG

In [ ]:
import os
import tensorflow as tf
import keras

from tensorflow.keras import layers, models, backend, utils, metrics

In [ ]:
#코랩 마운트
#from google.colab import drive
#drive.mount('/content/drive/', force_remount=True)

In [ ]:
#%cd /content/drive/MyDrive/face

In [ ]:
#압축해제
#!unzip -qq "/content/drive/MyDrive/face/real-vs-fake.zip"

In [ ]:
#VGG import 및 입력크기 지정
from keras.applications.vgg16 import VGG16

INPUT_SIZE = 224
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(INPUT_SIZE, INPUT_SIZE,3))

In [ ]:
#가져와서(전이학습) 고정된 층 학습시키않도록 함
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
#구조 생성
from keras.models import Model
from keras.layers import Dense, Flatten

input_ = vgg16.input
output_ = vgg16(input_)
last_layer = Flatten(name='flatten')(output_)
last_layer = Dense(1, activation='sigmoid')(last_layer)
model = Model(inputs=input_, outputs=last_layer)

In [ ]:
# Define hyperparameters
INPUT_SIZE = 224 
BATCH_SIZE = 16
STEPS_PER_EPOCH = 200
EPOCHS = 10

In [ ]:
'''#개수 확인(14만개 사용)
from glob import glob
filepaths1 = list(glob('/content/drive/MyDrive/face/train/real/*.jpg'))
filepaths2 = list(glob('/content/drive/MyDrive/face/test/real/*.jpg'))
filepaths3 = list(glob('/content/drive/MyDrive/face/valid/real/*.jpg'))
print(len(filepaths1), len(filepaths2), len(filepaths3))'''

In [ ]:
#데이터 경로 지정 등 설정
from keras.preprocessing.image import ImageDataGenerator
path = '/content/drive/MyDrive/face/'

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

training_data_generator = ImageDataGenerator(rescale = 1./255)
validing_data_generator = ImageDataGenerator(rescale = 1./255)
testing_data_generator = ImageDataGenerator(rescale = 1./255)

train_generator = training_data_generator.flow_from_directory(
    path + 'train/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

valid_generator = validing_data_generator.flow_from_directory(
    path + 'valid/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_generator = testing_data_generator.flow_from_directory(
    path + 'test/',
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    shuffle = False,
    class_mode='binary'
)

In [ ]:
#모델 학습
train_steps = 100000//16
valid_steps = 20000//16

hist = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=EPOCHS,
    validation_data=valid_generator,
    validation_steps=valid_steps
)

In [ ]:
#acc 및 loss plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate([ "accuracy", "loss"]):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
model.save("VGG")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("VGG")

In [ ]:
#test 평가
model.evaluate(test_generator)

## DenseNet

In [ ]:
### densenet 
import os
import tensorflow as tf
import keras

from tensorflow.keras import layers, models, backend, utils, metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#코랩 마운트
#from google.colab import drive
#drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/face2

In [ ]:
#압축해제
!unzip -qq "/content/drive/MyDrive/r-vs-f.zip"

In [ ]:
#개수 확인(14만개 중 1/10사용)
from glob import glob
filepaths1 = list(glob('/content/drive/MyDrive/face/train/real/*.jpg'))
filepaths2 = list(glob('/content/drive/MyDrive/face/test/real/*.jpg'))
filepaths3 = list(glob('/content/drive/MyDrive/face/valid/real/*.jpg'))
print(len(filepaths1), len(filepaths2), len(filepaths3))

In [ ]:
#데이터 경로 지정 등 설정
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator()

path = '/content/drive/My Drive/face/'

train_generator = image_gen.flow_from_directory(
    path + 'train/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    class_mode='binary'
)

valid_generator = image_gen.flow_from_directory(
    path + 'valid/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    class_mode='binary'
)

test_generator = image_gen.flow_from_directory(
    path + 'test/',
    target_size=(224, 224),
    batch_size=64,
    color_mode='rgb',
    shuffle = False,
    class_mode='binary'
)

In [ ]:
#densenet import 및 모델링
from tensorflow.keras.applications import DenseNet121

densenet = DenseNet121( weights='imagenet', include_top=False, input_shape=(224,224,3) )

model = models.Sequential()

model = models.Sequential([ 
        densenet,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

In [ ]:
#모델 학습
train_steps = 10000//64
valid_steps = 2000//64

hist = model.fit(
    train_generator,
    epochs=20,
    validation_data=valid_generator,
    validation_steps=valid_steps
)

In [ ]:
#acc 및 loss plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate([ "accuracy", "loss"]):
    ax[i].plot(hist.history[met])
    ax[i].plot(hist.history["val_" + met])
    ax[i].set_title("Model {}".format(met))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(met)
    ax[i].legend(["train", "val"])

In [ ]:
#test 평가
model.evaluate(test_generator)

In [ ]:
#이미지 대입
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image

test_image = image.load_img('/content/drive/MyDrive/11/fake2.jpg', target_size=(224, 224, 3))
plt.imshow(test_image)

fix_img = image.img_to_array(test_image)
fix_img = np.expand_dims(test_image, axis=0)

In [ ]:
#사진이 real인지 fake인지 출력
preds = model.predict(fix_img)
pct = np.max(preds)
if preds < 0.5:
    print('The Prediction of the sample is: It Is fake')
else:
    print('Prediction is real')
plt.imshow(test_image)

In [ ]:
#혼동행렬
y_pred = model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(np.int)

plt.figure(figsize = (8,5))
sns.heatmap(metrics.confusion_matrix(test_generator.classes, y_pred.round()), annot = True,fmt="d",cmap = "Blues")
plt.show()

In [ ]:
#AUC
print("ROC-AUC Score:", metrics.roc_auc_score(test_generator.classes, y_pred))
print("AP Score:", metrics.average_precision_score(test_generator.classes, y_pred))

print(metrics.classification_report(test_generator.classes, y_pred > 0.5))